In [3]:
import openfermion as of

# Set Hubbard model parameters
x_dim = 2
y_dim = 3
tunneling = 1.0
coulomb = 4.0
spinless=True

# Create Hubbard model Hamiltonian as FermionOperator
hubbard_model = of.fermi_hubbard(x_dim, y_dim, tunneling, coulomb, spinless=spinless)
print(hubbard_model)

4.0 [0^ 0 1^ 1] +
4.0 [0^ 0 2^ 2] +
-1.0 [0^ 1] +
-1.0 [0^ 2] +
-1.0 [0^ 4] +
-1.0 [1^ 0] +
4.0 [1^ 1 3^ 3] +
-1.0 [1^ 3] +
-1.0 [1^ 5] +
-1.0 [2^ 0] +
4.0 [2^ 2 3^ 3] +
4.0 [2^ 2 4^ 4] +
-1.0 [2^ 3] +
-1.0 [2^ 4] +
-1.0 [3^ 1] +
-1.0 [3^ 2] +
4.0 [3^ 3 5^ 5] +
-1.0 [3^ 5] +
-1.0 [4^ 0] +
-1.0 [4^ 2] +
4.0 [4^ 4 0^ 0] +
4.0 [4^ 4 5^ 5] +
-1.0 [4^ 5] +
-1.0 [5^ 1] +
-1.0 [5^ 3] +
-1.0 [5^ 4] +
4.0 [5^ 5 1^ 1]


In [4]:
from scipy.sparse.linalg import expm_multiply

# Create a random initial state
n_qubits = of.count_qubits(hubbard_model)
initial_state = of.haar_random_vector(2**n_qubits, seed=7)

# Convert Hamiltonian to sparse matrix
hubbard_sparse = of.get_sparse_operator(of.jordan_wigner(hubbard_model))

# Set evolution time
time = 1.0

# Apply exp(-i H t) to the state
exact_state = expm_multiply(-1j*hubbard_sparse*time, initial_state)
print(hubbard_sparse)

  (2, 1)	(-1+0j)
  (4, 1)	(-1+0j)
  (16, 1)	(-1+0j)
  (1, 2)	(-1+0j)
  (8, 2)	(-1+0j)
  (32, 2)	(-1+0j)
  (3, 3)	(4+0j)
  (6, 3)	(1+0j)
  (9, 3)	(-1+0j)
  (18, 3)	(1+0j)
  (33, 3)	(-1+0j)
  (1, 4)	(-1+0j)
  (8, 4)	(-1+0j)
  (16, 4)	(-1+0j)
  (5, 5)	(4+0j)
  (6, 5)	(-1+0j)
  (9, 5)	(-1+0j)
  (17, 5)	(-1+0j)
  (20, 5)	(1+0j)
  (3, 6)	(1+0j)
  (5, 6)	(-1+0j)
  (10, 6)	(-1+0j)
  (12, 6)	(1+0j)
  (18, 6)	(-1+0j)
  (36, 6)	(1+0j)
  :	:
  (58, 57)	(-1+0j)
  (60, 57)	(-1+0j)
  (43, 58)	(-1+0j)
  (46, 58)	(1+0j)
  (54, 58)	(-1+0j)
  (57, 58)	(-1+0j)
  (58, 58)	(16+0j)
  (47, 59)	(1+0j)
  (55, 59)	(-1+0j)
  (59, 59)	(24+0j)
  (62, 59)	(1+0j)
  (30, 60)	(1+0j)
  (45, 60)	(-1+0j)
  (54, 60)	(1+0j)
  (57, 60)	(-1+0j)
  (60, 60)	(16+0j)
  (31, 61)	(1+0j)
  (55, 61)	(1+0j)
  (61, 61)	(24+0j)
  (62, 61)	(-1+0j)
  (47, 62)	(1+0j)
  (59, 62)	(1+0j)
  (61, 62)	(-1+0j)
  (62, 62)	(24+0j)
  (63, 63)	(36+0j)


In [5]:
import cirq
import openfermioncirq as ofc
import numpy as np

# Convert Hamiltonian to a DiagonalCoulombHamiltonian instance
hubbard_hamiltonian = of.get_diagonal_coulomb_hamiltonian(hubbard_model)

# Initialize qubits
qubits = cirq.LineQubit.range(n_qubits)

# Create circuit
circuit = cirq.Circuit.from_ops(
    ofc.simulate_trotter(
        qubits, hubbard_hamiltonian, time,
        n_steps=10,
        order=0,
        algorithm=ofc.trotter.LINEAR_SWAP_NETWORK)
)

# Apply the circuit to the initial state
result = circuit.apply_unitary_effect_to_state(initial_state)

# Compute the fidelity with the final state from exact evolution
fidelity = abs(np.dot(exact_state, result.conj()))**2

print(fidelity)
print(circuit)

0.9732473269809888
0: ───XXYY──────────YXXY───@──────────×ᶠ──────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────YXXY───@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────YXXY───@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────────Rz(-0.0π)───────────────────────────────────────────────────────────────────────XXYY──────────#2───────@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────#2───────@──────────×ᶠ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────XXYY──────────#2───────@──────────×ᶠ──────────────────────────────────────────────Rz(-0.0π)───────────────XXYY──────────YXXY───@──────────×ᶠ───────────────────────

In [6]:
# Create circuit
circuit = cirq.Circuit.from_ops(
    ofc.simulate_trotter(
        qubits, hubbard_hamiltonian, time,
        n_steps=1,
        order=0,
        algorithm=ofc.trotter.LINEAR_SWAP_NETWORK,
        omit_final_swaps=True),
    strategy=cirq.InsertStrategy.EARLIEST
)

cirq.DropNegligible().optimize_circuit(circuit)
print(circuit.to_text_diagram(transpose=True))

0    1            2            3            4            5
│    │            │            │            │            │
XXYY─XXYY^(-7/11) XXYY─────────XXYY^(-7/11) XXYY─────────XXYY^(-7/11)
│    │            │            │            │            │
@────@^(8/11)     @────────────@^(8/11)     @────────────@^(8/11)
│    │            │            │            │            │
×ᶠ───×ᶠ           ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ           │
│    │            │            │            │            │
XXYY─XXYY^(-7/11) │            │            XXYY─────────XXYY^(-7/11)
│    │            │            │            │            │
@────@^(8/11)     │            │            @────────────@^(8/11)
│    │            │            │            │            │
×ᶠ───×ᶠ           ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    XXYY─────────

In [7]:
xmon_circuit = cirq.google.optimized_for_xmon(circuit)

print(xmon_circuit.to_text_diagram(transpose=True))

0                      1                      2                      3                      4                      5
│                      │                      │                      │                      │                      │
PhasedX(-0.614)^0.5    │                      PhasedX(0.0784)^0.5    │                      PhasedX(-2.22e-15)^0.5 │
│                      │                      │                      │                      │                      │
@──────────────────────@                      @──────────────────────@                      @──────────────────────@
│                      │                      │                      │                      │                      │
PhasedX(0.386)^0.5     PhasedX(0.886)^(9/11)  PhasedX(-0.922)^0.5    PhasedX(-0.422)^(9/11) PhasedX(1.0)^0.5       Y^(-9/11)
│                      │                      │                      │                      │                      │
@──────────────────────@                      @─────────

In [8]:
two_qubit_gates = xmon_circuit.findall_operations_with_gate_type(cirq.TwoQubitGate)
print(len(list(two_qubit_gates)))

39


In [9]:
ansatz = ofc.SwapNetworkTrotterAnsatz(
    hubbard_hamiltonian,
    iterations=3)
print(ansatz.circuit.to_text_diagram(transpose=True))

0    1            2            3            4            5
│    │            │            │            │            │
XXYY─XXYY^T_0_1_0 XXYY─────────XXYY^T_2_3_0 XXYY─────────XXYY^T_4_5_0
│    │            │            │            │            │
@────@^V_0_1_0    @────────────@^V_2_3_0    @────────────@^V_4_5_0
│    │            │            │            │            │
×ᶠ───×ᶠ           ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    ×ᶠ───────────×ᶠ           ×ᶠ───────────×ᶠ           │
│    │            │            │            │            │
XXYY─XXYY^T_1_3_0 ×ᶠ───────────×ᶠ           XXYY─────────XXYY^T_2_4_0
│    │            │            │            │            │
@────@^V_1_3_0    │            │            @────────────@^V_2_4_0
│    │            │            │            │            │
×ᶠ───×ᶠ           │            │            ×ᶠ───────────×ᶠ
│    │            │            │            │            │
│    XXYY───────

In [10]:
import openfermion as of
# Set Hubbard model parameters
x_dim = 2
y_dim = 3
tunneling = 1.0
coulomb = 4.0
spinless=True

# Create Hubbard model Hamiltonian as FermionOperator
hubbard_model = of.fermi_hubbard(x_dim, y_dim, tunneling, coulomb, spinless=spinless)
print(hubbard_model)

from openfermion.transforms import get_sparse_operator
n_electrons = 2

# Compute the ground state energy at the specified number of electrons
hamiltonian_sparse = get_sparse_operator(hubbard_model)
true_ground_energy, _ = of.jw_get_ground_state_at_particle_number(
    hamiltonian_sparse, n_electrons)

print(true_ground_energy)



4.0 [0^ 0 1^ 1] +
4.0 [0^ 0 2^ 2] +
-1.0 [0^ 1] +
-1.0 [0^ 2] +
-1.0 [0^ 4] +
-1.0 [1^ 0] +
4.0 [1^ 1 3^ 3] +
-1.0 [1^ 3] +
-1.0 [1^ 5] +
-1.0 [2^ 0] +
4.0 [2^ 2 3^ 3] +
4.0 [2^ 2 4^ 4] +
-1.0 [2^ 3] +
-1.0 [2^ 4] +
-1.0 [3^ 1] +
-1.0 [3^ 2] +
4.0 [3^ 3 5^ 5] +
-1.0 [3^ 5] +
-1.0 [4^ 0] +
-1.0 [4^ 2] +
4.0 [4^ 4 0^ 0] +
4.0 [4^ 4 5^ 5] +
-1.0 [4^ 5] +
-1.0 [5^ 1] +
-1.0 [5^ 3] +
-1.0 [5^ 4] +
4.0 [5^ 5 1^ 1]
-3.1231056256176655


In [11]:
import openfermion as of

In [12]:
x_dim = 2
y_dim = 3
tunneling = 1.0
coulomb = 4.0
spinless=True
# Create Hubbard model Hamiltonian as FermionOperator
hubbard_model = of.fermi_hubbard(x_dim, y_dim, tunneling, coulomb, spinless=spinless)
print(hubbard_model)

4.0 [0^ 0 1^ 1] +
4.0 [0^ 0 2^ 2] +
-1.0 [0^ 1] +
-1.0 [0^ 2] +
-1.0 [0^ 4] +
-1.0 [1^ 0] +
4.0 [1^ 1 3^ 3] +
-1.0 [1^ 3] +
-1.0 [1^ 5] +
-1.0 [2^ 0] +
4.0 [2^ 2 3^ 3] +
4.0 [2^ 2 4^ 4] +
-1.0 [2^ 3] +
-1.0 [2^ 4] +
-1.0 [3^ 1] +
-1.0 [3^ 2] +
4.0 [3^ 3 5^ 5] +
-1.0 [3^ 5] +
-1.0 [4^ 0] +
-1.0 [4^ 2] +
4.0 [4^ 4 0^ 0] +
4.0 [4^ 4 5^ 5] +
-1.0 [4^ 5] +
-1.0 [5^ 1] +
-1.0 [5^ 3] +
-1.0 [5^ 4] +
4.0 [5^ 5 1^ 1]


In [13]:
from openfermion.transforms import get_sparse_operator
hamiltonian_sparse = get_sparse_operator(hubbard_model)
print(hamiltonian_sparse)

  (2, 1)	(-1+0j)
  (4, 1)	(-1+0j)
  (16, 1)	(-1+0j)
  (1, 2)	(-1+0j)
  (8, 2)	(-1+0j)
  (32, 2)	(-1+0j)
  (3, 3)	(4+0j)
  (6, 3)	(1+0j)
  (9, 3)	(-1+0j)
  (18, 3)	(1+0j)
  (33, 3)	(-1+0j)
  (1, 4)	(-1+0j)
  (8, 4)	(-1+0j)
  (16, 4)	(-1+0j)
  (5, 5)	(4+0j)
  (6, 5)	(-1+0j)
  (9, 5)	(-1+0j)
  (17, 5)	(-1+0j)
  (20, 5)	(1+0j)
  (3, 6)	(1+0j)
  (5, 6)	(-1+0j)
  (10, 6)	(-1+0j)
  (12, 6)	(1+0j)
  (18, 6)	(-1+0j)
  (36, 6)	(1+0j)
  :	:
  (58, 57)	(-1+0j)
  (60, 57)	(-1+0j)
  (43, 58)	(-1+0j)
  (46, 58)	(1+0j)
  (54, 58)	(-1+0j)
  (57, 58)	(-1+0j)
  (58, 58)	(16+0j)
  (47, 59)	(1+0j)
  (55, 59)	(-1+0j)
  (59, 59)	(24+0j)
  (62, 59)	(1+0j)
  (30, 60)	(1+0j)
  (45, 60)	(-1+0j)
  (54, 60)	(1+0j)
  (57, 60)	(-1+0j)
  (60, 60)	(16+0j)
  (31, 61)	(1+0j)
  (55, 61)	(1+0j)
  (61, 61)	(24+0j)
  (62, 61)	(-1+0j)
  (47, 62)	(1+0j)
  (59, 62)	(1+0j)
  (61, 62)	(-1+0j)
  (62, 62)	(24+0j)
  (63, 63)	(36+0j)


In [14]:
n_electrons = 2
true_ground_energy, _ = of.jw_get_ground_state_at_particle_number(hamiltonian_sparse, n_electrons)

print(true_ground_energy)

-3.12310562561766


In [15]:
import openfermion

diatomic_bond_length = .7414
geometry = [('H', (0., 0., 0.)), 
            ('H', (0., 0., diatomic_bond_length))]
basis = 'sto-3g'
multiplicity = 1
charge = 0
description = format(diatomic_bond_length)

molecule = openfermion.MolecularData(
    geometry,
    basis,
    multiplicity,
    description=description)
molecule.load()

hamiltonian = molecule.get_molecular_hamiltonian()
print("Bond Length in Angstroms: {}".format(diatomic_bond_length))
print("Hartree Fock (mean-field) energy in Hartrees: {}".format(molecule.hf_energy))
print("FCI (Exact) energy in Hartrees: {}".format(molecule.fci_energy))

Bond Length in Angstroms: 0.7414
Hartree Fock (mean-field) energy in Hartrees: -1.116684386906734
FCI (Exact) energy in Hartrees: -1.137270174625328


In [16]:
import cirq
import openfermioncirq
import sympy

class MyAnsatz(openfermioncirq.VariationalAnsatz):
    
    def params(self):
        """The parameters of the ansatz."""
        return [sympy.Symbol('theta_0')]
    
    def operations(self, qubits):
        """Produce the operations of the ansatz circuit."""
        q0, q1, q2, q3 = qubits
        yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
        yield cirq.XPowGate(exponent=-0.5).on(q3)
        
        yield cirq.CNOT(q0, q1), cirq.CNOT(q1, q2), cirq.CNOT(q2, q3)
        yield cirq.ZPowGate(exponent=sympy.Symbol('theta_0')).on(q3)
        yield cirq.CNOT(q2, q3), cirq.CNOT(q1, q2), cirq.CNOT(q0, q1)
        
        yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
        yield cirq.XPowGate(exponent=0.5).on(q3)

    def _generate_qubits(self):
        """Produce qubits that can be used by the ansatz circuit."""
        return cirq.LineQubit.range(4)

In [18]:
ansatz = MyAnsatz()
objective = openfermioncirq.HamiltonianObjective(hamiltonian)
q0, q1, _, _ = ansatz.qubits
preparation_circuit = cirq.Circuit.from_ops(
    cirq.X(q0),
    cirq.X(q1))
study = openfermioncirq.VariationalStudy(
    name='my_hydrogen_study',
    ansatz=ansatz,
    objective=objective,
    preparation_circuit=preparation_circuit)
print(study.circuit)
print(study.num_params)

0: ───X───H────────@───────────────────────────────────@───H───
                   │                                   │
1: ───X───H────────X───@───────────────────────@───────X───H───
                       │                       │
2: ───────H────────────X───@───────────────@───X───────H───────
                           │               │
3: ───────X^-0.5───────────X───Z^theta_0───X───X^0.5───────────
1


In [19]:
# Perform optimization.
import numpy
import openfermioncirq as ofc
from openfermioncirq.optimization import COBYLA, OptimizationParams
algorithm=ofc.optimization.ScipyOptimizationAlgorithm(kwargs={'method': 'COBYLA'},uses_bounds=False)
optimization_params = OptimizationParams(algorithm=algorithm,initial_guess=[0.1])
result = study.optimize(optimization_params)




TypeError: ufunc 'multiply' output (typecode 'O') could not be coerced to provided output parameter (typecode 'D') according to the casting rule ''same_kind''